In [1]:

%load_ext autoreload
%autoreload 2


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

In [3]:
train_dataset, val_dataset, train_loader, val_loader = CIFAR10()

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"



# train standard model todo: add more epochs maybe 100

In [6]:
from torch.utils.tensorboard import SummaryWriter
net_ori = BaseNet().to(device)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_ori.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base.pth'
device = "cuda"
writer = SummaryWriter("NormalBase")

In [7]:
train(net_ori, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 50, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/50]	
Epoch [1/50]	 Training Loss: 2.3042824304927034	 lr: 0.001
Epoch [1/50]	 Validation Loss: 2.3028613162946097	 lr: 0.001
Epoch [1/50]	
Epoch [2/50]	 Training Loss: 2.3023489064267832	 lr: 0.001
Epoch [2/50]	 Validation Loss: 2.301627551453023	 lr: 0.001
Epoch [2/50]	
Epoch [3/50]	 Training Loss: 2.300663931595395	 lr: 0.001
Epoch [3/50]	 Validation Loss: 2.2992500685438326	 lr: 0.001
Epoch [3/50]	
Epoch [4/50]	 Training Loss: 2.2961477519910964	 lr: 0.001
Epoch [4/50]	 Validation Loss: 2.291310117214541	 lr: 0.001
Epoch [4/50]	
Epoch [5/50]	 Training Loss: 2.274019359017882	 lr: 0.001
Epoch [5/50]	 Validation Loss: 2.2399823062027555	 lr: 0.001
Epoch [5/50]	
Epoch [6/50]	 Training Loss: 2.2003042868641027	 lr: 0.001
Epoch [6/50]	 Validation Loss: 2.1800561916979055	 lr: 0.001
Epoch [6/50]	
Epoch [7/50]	 Training Loss: 2.163083449044191	 lr: 0.001
Epoch [7/50]	 Validation Loss: 2.1463266475291194	 lr: 0.001
Epoch [7/50]	
Epoch [8/50]	 Training Loss: 2.1253100228126702	 lr: 

In [8]:
#test clean accuray
test(val_loader, net_ori)


Accuracy of the network on the 10000 test images: 56 %


In [9]:
#net = BaseNet().to(device)
#test attacks 
pgd_attack = generate_adv(net_ori, "pgd")
fgsm_attack = generate_adv(net_ori, "fgsm")
nifgsm_attack = generate_adv(net_ori, "nifgsm")
vmifgsm_attack = generate_adv(net_ori, "vmifgsm")




In [10]:
images, labels = load_dataset(10000, val_loader)

In [11]:
adv_images_pgd_adv = pgd_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 4.03 %
adv model


In [12]:
adv_images_fgsm_adv = fgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 7.43 %
adv model


In [13]:
#nifgsm_attack = generate_adv(net, "nifgsm")

adv_images_nifgsm_adv = nifgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 5.60 %
adv model


In [14]:
#vmifgsm_attack = generate_adv(net, "vmifgsm")
adv_images_vmifgsm_adv =vmifgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 4.33 %
adv model


# Adversarial Training

In [15]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(net_ori) #am I doing correct? 
for param in net_adv.parameters():
    print(param)

Parameter containing:
tensor([[[[ 5.9848e-02, -5.2299e-02,  2.1430e-01,  3.9787e-02,  2.1806e-01],
          [-5.0105e-02, -1.4161e-01, -3.0649e-02,  7.1707e-02,  6.1908e-02],
          [-8.4976e-02, -6.6476e-02, -2.0944e-01, -1.3324e-01, -8.4117e-02],
          [-7.0177e-03, -1.8230e-01, -5.3974e-02, -1.3035e-01,  2.1496e-02],
          [ 2.4488e-01, -4.5639e-02,  3.4719e-02, -5.2099e-02,  1.1870e-01]],

         [[ 1.2060e-02, -1.2220e-02,  1.3480e-01, -1.0539e-02,  1.5150e-01],
          [-1.0929e-01, -1.1086e-01, -9.1473e-02,  2.9505e-03,  1.0279e-01],
          [ 9.7334e-02, -1.1177e-01, -1.7327e-01,  7.1333e-02,  9.2015e-03],
          [ 1.2965e-02, -9.0692e-02, -1.2689e-01,  8.6416e-02, -1.2764e-01],
          [ 2.6695e-01,  6.7538e-02,  1.2677e-01,  1.7106e-01,  1.2766e-01]],

         [[-2.9045e-02, -1.2880e-01,  2.8464e-02,  1.3241e-01,  1.3513e-01],
          [ 1.0995e-01, -5.6772e-02, -2.2852e-02,  1.1513e-01, -3.6542e-02],
          [ 1.1706e-02, -3.7067e-02, -5.8166e-02, 

In [16]:

    
    
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

In [17]:
train(net_adv, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 50, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/50]	
Epoch [1/50]	 Training Loss: 2.2839574600424606	 lr: 0.001
Epoch [1/50]	 Validation Loss: 1.6474633835539032	 lr: 0.001
Epoch [1/50]	
Epoch [2/50]	 Training Loss: 2.0984043212193053	 lr: 0.001
Epoch [2/50]	 Validation Loss: 1.6317872488045995	 lr: 0.001
Epoch [2/50]	
Epoch [3/50]	 Training Loss: 2.057575298087371	 lr: 0.001
Epoch [3/50]	 Validation Loss: 1.6104123969621296	 lr: 0.001
Epoch [3/50]	
Epoch [4/50]	 Training Loss: 2.0349581461123494	 lr: 0.001
Epoch [4/50]	 Validation Loss: 1.607916465288476	 lr: 0.001
Epoch [4/50]	
Epoch [5/50]	 Training Loss: 2.0203371078461942	 lr: 0.001
Epoch [5/50]	 Validation Loss: 1.6162802388396444	 lr: 0.001
Epoch [5/50]	
Epoch [6/50]	 Training Loss: 2.0091663547184155	 lr: 0.001
Epoch [6/50]	 Validation Loss: 1.6130752729464182	 lr: 0.001
Epoch [6/50]	
Epoch [7/50]	 Training Loss: 2.0004410115654205	 lr: 0.001
Epoch [7/50]	 Validation Loss: 1.6110188281988795	 lr: 0.001
Epoch [7/50]	
Epoch [8/50]	 Training Loss: 1.9943394011548718	 l

In [18]:
#test clean accuray
test(val_loader, net_adv)

Accuracy of the network on the 10000 test images: 47 %


In [19]:
acc = clean_accuracy(net_adv, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 38.64 %
adv model


In [20]:
acc = clean_accuracy(net_adv, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 38.14 %
adv model


In [21]:
acc = clean_accuracy(net_adv, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 38.71 %
adv model


In [22]:
acc = clean_accuracy(net_adv, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 38.57 %
adv model


# TODO: Adversarial Training by nifgsm

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv_ni = copy.deepcopy(net_ori) #am I doing correct? 
for param in net_adv_ni.parameters():
    print(param)

In [ ]:

    
    
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv_ni.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_nifgsm.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_nifgsm")
ni_attack = generate_adv(net_adv_ni, "nifgsm")

In [ ]:
train(net_adv_ni, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 50, writer, 
                 PATH, True, ni_attack, device="cuda",
                  n_steps_show=1000)

# TODO: Adversarial Training by vmifgsm

# TODO: Adversarial Training by fgsm